In [1]:
# pip install --upgrade pip

In [2]:
# %%capture
# %pip install -U transformers
# %pip install -U datasets
# %pip install -U accelerate
# %pip install -U peft
%pip install -U trl --quiet
%pip install -U bitsandbytes --quiet
%pip install evaluate bert-score rouge_score --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.0/348.0 kB 7.3 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.7 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.

In [3]:
%%capture
%pip install rouge_score

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from peft import  LoraConfig, get_peft_model
import os, torch, wandb
from datasets import load_dataset
from trl import SFTTrainer, setup_chat_format, SFTConfig

2025-05-14 19:54:22.166977: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747252462.350176      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747252462.402006      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
import bitsandbytes as bnb
from sklearn.metrics import f1_score
import numpy as np
import torch
from typing import Dict
from transformers import EvalPrediction
from evaluate import load
from bert_score import score as bert_score

In [6]:
from huggingface_hub import notebook_login

# notebook_login()

In [7]:
from huggingface_hub import login
tokn="add_token"
login(token = tokn) # add your token from huggingface

In [8]:
os.environ["WANDB_DISABLED"] = "true"

In [9]:
base_model = "meta-llama/Llama-3.2-3B" # To use this form, you must grant access (enter your info, they will grant access within 10 minutes)

# Quantization

In [10]:
# quantization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    # llm_int8_enable_fp32_cpu_offload=True
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load model with quantization
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map={"": torch.cuda.current_device()},  # ✅ Force full model to GPU
)

config.json:   0%|          | 0.00/844 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

# Tokenizer

In [11]:
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

# Load Dataset & preprocessing

In [12]:
torch.cuda.empty_cache()

In [13]:
# dataset = load_dataset("gsm8k", "main", split="train[:5000]")
# dataset = load_dataset("hendrycks/math")
# dataset = load_dataset("stanfordnlp/proofwriter", "depth-3")
# dataset = load_dataset("hendrycks/competition_math")
dataset = load_dataset("deepmind/math_dataset", "arithmetic__add_or_sub", split="train[:5000]")

README.md:   0%|          | 0.00/24.8k [00:00<?, ?B/s]

math_dataset.py:   0%|          | 0.00/8.40k [00:00<?, ?B/s]

The repository for deepmind/math_dataset contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/deepmind/math_dataset.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


Generating train split:   0%|          | 0/1999998 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [14]:
print("ValueError: BuilderConfig 'algebra' not found. Available: ['algebra__linear_1d', 'algebra__linear_1d_composed', 'algebra__linear_2d', 'algebra__linear_2d_composed', 'algebra__polynomial_roots', 'algebra__polynomial_roots_composed', 'algebra__sequence_next_term', 'algebra__sequence_nth_term', 'arithmetic__add_or_sub', 'arithmetic__add_or_sub_in_base', 'arithmetic__add_sub_multiple', 'arithmetic__div', 'arithmetic__mixed', 'arithmetic__mul', 'arithmetic__mul_div_multiple', 'arithmetic__nearest_integer_root', 'arithmetic__simplify_surd', 'calculus__differentiate', 'calculus__differentiate_composed', 'comparison__closest', 'comparison__closest_composed', 'comparison__kth_biggest', 'comparison__kth_biggest_composed', 'comparison__pair', 'comparison__pair_composed', 'comparison__sort', 'comparison__sort_composed', 'measurement__conversion', 'measurement__time', 'numbers__base_conversion', 'numbers__div_remainder', 'numbers__div_remainder_composed', 'numbers__gcd', 'numbers__gcd_composed', 'numbers__is_factor', 'numbers__is_factor_composed', 'numbers__is_prime', 'numbers__is_prime_composed', 'numbers__lcm', 'numbers__lcm_composed', 'numbers__list_prime_factors', 'numbers__list_prime_factors_composed', 'numbers__place_value', 'numbers__place_value_composed', 'numbers__round_number', 'numbers__round_number_composed', 'polynomials__add', 'polynomials__coefficient_named', 'polynomials__collect', 'polynomials__compose', 'polynomials__evaluate', 'polynomials__evaluate_composed', 'polynomials__expand', 'polynomials__simplify_power', 'probability__swr_p_level_set', 'probability__swr_p_sequence']")

ValueError: BuilderConfig 'algebra' not found. Available: ['algebra__linear_1d', 'algebra__linear_1d_composed', 'algebra__linear_2d', 'algebra__linear_2d_composed', 'algebra__polynomial_roots', 'algebra__polynomial_roots_composed', 'algebra__sequence_next_term', 'algebra__sequence_nth_term', 'arithmetic__add_or_sub', 'arithmetic__add_or_sub_in_base', 'arithmetic__add_sub_multiple', 'arithmetic__div', 'arithmetic__mixed', 'arithmetic__mul', 'arithmetic__mul_div_multiple', 'arithmetic__nearest_integer_root', 'arithmetic__simplify_surd', 'calculus__differentiate', 'calculus__differentiate_composed', 'comparison__closest', 'comparison__closest_composed', 'comparison__kth_biggest', 'comparison__kth_biggest_composed', 'comparison__pair', 'comparison__pair_composed', 'comparison__sort', 'comparison__sort_composed', 'measurement__conversion', 'measurement__time', 'numbers__base_conversion', 'numbers__div_remainder', 'numbers__div_remainder_composed', 'numbers__gcd', 'numbers__gcd_composed', 'n

In [15]:
dataset, dataset[79]

(Dataset({
     features: ['question', 'answer'],
     num_rows: 5000
 }),
 {'question': "b'Work out -20.33 + -707130.\\n'",
  'answer': "b'-707150.33\\n'"})

In [16]:
import ast

def clean_byte_like_strings(row):
    try:
        row["question"] = ast.literal_eval(row["question"]).strip()
    except:
        row["question"] = row["question"].strip()

    try:
        row["answer"] = ast.literal_eval(row["answer"]).strip()
    except:
        row["answer"] = row["answer"].strip()
        
    return row


In [17]:
dataset = dataset.map(clean_byte_like_strings)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [18]:
dataset, dataset[79]

(Dataset({
     features: ['question', 'answer'],
     num_rows: 5000
 }),
 {'question': 'Work out -20.33 + -707130.', 'answer': '-707150.33'})

In [19]:
llama_template = """<|begin_of_text|>
{% for message in messages %}
<|start_header_id|>{{ message['role'] }}<|end_header_id|>
{{ message['content'] }}<|eot_id|>
{% endfor %}"""

def format_chat_template(row):
    instruction = "You are a helpful math tutor."
    messages = [
        {"role": "system", "content": instruction},
        {"role": "user", "content": row["question"]},
        {"role": "assistant", "content": row["answer"].split("####")[-1].strip()}
    ]

    row["text"] = tokenizer.apply_chat_template(
        messages,
        chat_template=llama_template,
        tokenize=False
    )
    return row

In [20]:
dataset = dataset.map(format_chat_template,num_proc= 4,)

Map (num_proc=4):   0%|          | 0/5000 [00:00<?, ? examples/s]

In [21]:
# dataset = dataset.select(range(4000))

In [22]:
split_dataset = dataset.train_test_split(test_size=0.1, seed=42) # split data to train & test

train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]

In [23]:
def find_all_linear_names(model):
    cls = bnb.nn.Linear4bit
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names:  # needed for 16 bit
        lora_module_names.remove('lm_head')
    return list(lora_module_names)

modules = find_all_linear_names(model)

In [24]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 3072)
    (layers): ModuleList(
      (0-27): 28 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (k_proj): Linear4bit(in_features=3072, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=3072, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=3072, out_features=8192, bias=False)
          (up_proj): Linear4bit(in_features=3072, out_features=8192, bias=False)
          (down_proj): Linear4bit(in_features=8192, out_features=3072, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((3072,)

In [25]:
modules

['v_proj', 'o_proj', 'up_proj', 'down_proj', 'q_proj', 'gate_proj', 'k_proj']

# LoRA configration

In [26]:
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM", # this means that the model will generate the answer
    target_modules=modules
)
model, tokenizer = setup_chat_format(model, tokenizer)
model = get_peft_model(model, peft_config)

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


# Training setup

In [27]:
tokenizer.truncation_side = "left" # important note: this line is used based on your data template.
tokenizer.model_max_length = 1024

In [28]:
training_arguments = SFTConfig(
    output_dir="outputs",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    num_train_epochs=1,
    # eval_strategy ="epoch",
    warmup_steps=10,
    learning_rate=2e-4,
    fp16=True,
    bf16=False,
    group_by_length=True,
    report_to="none",
    max_length=512,
)

In [29]:
# SFTTrainer parameters setup
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    peft_config=peft_config,
    args=training_arguments,
    # compute_metrics=compute_metrics
)

Converting train dataset to ChatML:   0%|          | 0/4500 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/4500 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/4500 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/4500 [00:00<?, ? examples/s]

Converting eval dataset to ChatML:   0%|          | 0/500 [00:00<?, ? examples/s]

Adding EOS to eval dataset:   0%|          | 0/500 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/500 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/500 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [30]:
trainer.train()

/usr/local/lib/python3.11/dist-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


Step,Training Loss
500,2.099200


/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


TrainOutput(global_step=562, training_loss=2.0870045020486963, metrics={'train_runtime': 817.9301, 'train_samples_per_second': 5.502, 'train_steps_per_second': 0.687, 'total_flos': 3449816007303168.0, 'train_loss': 2.0870045020486963})

# Evaluation metrics

In [31]:
import torch
import gc
from tqdm import tqdm
from evaluate import load
from bert_score import score as bert_score

In [32]:
def simple_f1(pred, label):
    pred_tokens = pred.split()
    label_tokens = label.split()
    common = set(pred_tokens) & set(label_tokens)

    if len(common) == 0:
        return 0.0

    precision = len(common) / len(pred_tokens)
    recall = len(common) / len(label_tokens)

    if precision + recall == 0:
        return 0.0

    return 2 * (precision * recall) / (precision + recall)


In [33]:

rouge = load("rouge")

def evaluate_model(model, tokenizer, dataset, max_new_tokens=100, batch_size=16):
    model.eval()
    results = []
    f1_scores = []
    rouge_preds = []
    rouge_labels = []

    for i in tqdm(range(0, len(dataset), batch_size)):
        batch = dataset[i:i+batch_size]

        tokenizer.padding_side = "left"
        inputs = tokenizer(
            batch["text"],
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=tokenizer.model_max_length,
        ).to(model.device)

        with torch.no_grad():
            outputs = model.generate(
                input_ids=inputs["input_ids"],
                attention_mask=inputs["attention_mask"],
                max_new_tokens=max_new_tokens,
                do_sample=False,
            )

        decoded_preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        decoded_labels = batch["answer"]  # أو العمود الصحيح حسب الداتا

        # Clean up
        for pred, label in zip(decoded_preds, decoded_labels):
            pred_clean = pred.strip()
            label_clean = label.strip()

            # F1
            f1 = simple_f1(pred_clean, label_clean)
            f1_scores.append(f1)

            # ROUGE inputs
            rouge_preds.append(pred_clean)
            rouge_labels.append(label_clean)

            results.append({
                "prediction": pred_clean,
                "reference": label_clean,
                "f1": f1,
            })

        del inputs, outputs
        torch.cuda.empty_cache()
        gc.collect()

    # F1
    avg_f1 = sum(f1_scores) / len(f1_scores)

    # ROUGE
    rouge_result = rouge.compute(predictions=rouge_preds, references=rouge_labels)
    rougeL = rouge_result["rougeL"]

    # BERTScore
    P, R, F1 = bert_score(rouge_preds, rouge_labels, lang="en", device="cpu", verbose=False)
    avg_bert_f1 = F1.mean().item()

    print(f"\n Average F1 Score:     {avg_f1:.4f}")
    print(f" ROUGE-L Score:        {rougeL:.4f}")
    print(f" BERTScore (F1):       {avg_bert_f1:.4f}")

    return {
        "f1": avg_f1,
        "rougeL": rougeL,
        "bertscore_f1": avg_bert_f1,
        "results": results
    }


In [34]:
metrics = evaluate_model(model, tokenizer, eval_dataset, max_new_tokens=60)

  0%|          | 0/32 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
100%|██████████| 32/32 [06:15<00:00, 11.72s/it]


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



 Average F1 Score:     0.1271
 ROUGE-L Score:        0.1932
 BERTScore (F1):       0.8695


In [35]:
print(metrics['results'][121]['prediction'])
print("___________")
print(metrics['results'][121]['reference'])

system
You are a helpful math tutor.
user
Add -0.2 and -4.936803.
assistant
-5.136803
___________
-5.136803


In [36]:
print(metrics)

{'f1': 0.12710924001179422, 'rougeL': 0.19323383043087355, 'bertscore_f1': 0.8695175647735596, 'results': [{'prediction': 'system\nYou are a helpful math tutor.\nuser\nWhat is -650824 minus -2.6?\nassistant\n-650821.4', 'reference': '-650821.4', 'f1': 0.125}, {'prediction': 'system\nYou are a helpful math tutor.\nuser\nSubtract 0.15831811 from 0.\nassistant\n-0.15831811', 'reference': '-0.15831811', 'f1': 0.13333333333333333}, {'prediction': 'system\nYou are a helpful math tutor.\nuser\nSum 0.138404516 and 43.\nassistant\n43.138404516', 'reference': '43.138404516', 'f1': 0.13333333333333333}, {'prediction': 'system\nYou are a helpful math tutor.\nuser\nWhat is -249814083 plus -0.4?\nassistant\n-249814083.4', 'reference': '-249814083.4', 'f1': 0.125}, {'prediction': 'system\nYou are a helpful math tutor.\nuser\nWhat is -23.35 take away 0.3379?\nassistant\n-23.6879', 'reference': '-23.6879', 'f1': 0.11764705882352941}, {'prediction': 'system\nYou are a helpful math tutor.\nuser\n0.1 + -2